In [ ]:
import findspark
findspark.init("/opt/cloudera/parcels/CDH-6.3.1-1.cdh6.3.1.p0.1470567/lib/spark")

import os
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_181-cloudera"

from pyspark import SparkContext
from pyspark.sql import SparkSession,HiveContext,Window
from pyspark.sql import functions as fn
from pyspark.sql.types import IntegerType, FloatType, DoubleType, ArrayType, StringType, DecimalType,MapType

spark_session = SparkSession.builder.enableHiveSupport().appName("test").config("spark.driver.memory","30g").getOrCreate()
hc = HiveContext(spark_session.sparkContext)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

### Read Data

In [ ]:
# Load all tags of those who are tagged custumers status of MG
# Code to create table tmp_mg_tags_0513 please refer to SQL archived documents.
mg_tags = hc.sql("select * from rdtwarehouse.tmp_mg_tags_0513").toPandas()

In [ ]:
# Load dictionary of tags
ma_tags = hc.sql("select * from rdtwarehouse.cdp_ma_tags_0513").toPandas()

# To do some transformation of MG tags
split = ma_tags['tag_name'].str.split('/')
ma_tags['tag'] = pd.DataFrame([i[-1] if len(i) <= 3 else i[-3] if i[-3] == '个人地址' else i[-2] for i in split])
ma_tags['tag_value'] = pd.DataFrame([i[-1] for i in split])
ma_tags['tag_brand'] = pd.DataFrame([i[3] if len(i) >= 4 else '' for i in split])

In [ ]:
# Merge tag_id and tag_name
tags_c = mg_tags.merge(ma_tags, how = 'left', on = 'tag_id')
mg_tags_c = tags_c[(tags_c['tag_brand'] != 'ROEWE') & (tags_c['tag_brand'] != 'ROEWE专有')]

In [ ]:
mg_tags_c['tag'].unique()

In [ ]:
# load background information
city_level = pd.read_excel('city_level.xlsx', delimiter = ',')

### Basic Properities of all MG Customers

In [ ]:
# Total MG Customers
print('Total MG Customer:', len(mg_tags_c['id'].unique()))
ttl_mbr = len(mg_tags_c['id'].unique())

In [ ]:
# null% of customer age
print('客户寿命:',1-(len(mg_tags_c[(mg_tags_c['tag'] == '客户综合信息') & (mg_tags_c['tag_value'] == '客户寿命')])/ttl_mbr))

In [ ]:
# Distribution of MG customers' life age
pd.cut(mg_tags_c[(mg_tags_c['tag'] == '客户综合信息') & (mg_tags_c['tag_value'] == '客户寿命')]['score'].values,\
bins=[0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 1000]).value_counts()

In [ ]:
# Distribution of dealed customers
pd.cut(mg_tags_c[mg_tags_c['tag_value'] == '成交日期']['score'].values,\
bins=[20041231, 20051231, 20061231, 20071231, 20081231, 20091231, 20101231, 20111231, 20121231, 20131231, 20141231, 20151231,\
     20161231, 20171231, 20181231, 20191231, 20201231]).value_counts()

### Analysis of null rate of current CDP tags on MG Customers in last 6m

In [ ]:
# MG Customers last 6m
mg_cust_6m = mg_tags_c[mg_tags_c['id'].isin(mg_tags_c[(mg_tags_c['tag_value'] == '客户寿命') & (mg_tags_c['score'] <= 180)]['id'])]
mg_cust_6m['split'] = mg_cust_6m['tag_name'].str.split('/')

In [ ]:
print('MG Customer Last 6m:',len(mg_cust_6m['id'].unique()))

In [ ]:
# Customer stage
customer_stage = pd.DataFrame(mg_cust_6m[mg_cust_6m['tag'] == '客户状态']['tag_value'].value_counts())
customer_stage['vol%'] = customer_stage['tag_value']/len(mg_cust_6m['id'].unique())
print(customer_stage)

In [ ]:
# Calculate vol of selected tags and group by customer status
def null_rate(data, tag_name):
    targeted_tag_id = data[data['tag_name'] == tag_name]
    if len(targeted_tag_id) == 0:
        null_list = ["Empty Tags"]
    else:
        targeted = data[data['id'].isin(targeted_tag_id['id'])]
        null_list = pd.DataFrame(targeted[targeted['tag'] == '客户状态']['tag_value'].value_counts())
    return tag_name, null_list

def null_rate_contain(data,tag_name):
    mg_tags_dna = pd.DataFrame(data[['id','tag_name']].dropna())
    targeted_tag_id = data[data['tag_name'].str.contains(tag_name)]
    if len(targeted_tag_id) == 0:
        null_list = ["Empty Tags"]
    else:
        targeted = data[data['id'].isin(targeted_tag_id['id'])]
        null_list = pd.DataFrame(targeted[targeted['tag'] == '客户状态']['tag_value'].value_counts())
    return tag_name, null_list

In [ ]:
tag_name_leaf = ['全局标签/客户标签/行为属性标签/MG/APP/是否注册APP',\
            '全局标签/客户标签/行为属性标签/MG/官网/浏览官网/最后一次浏览官网日期',\
            '全局标签/客户标签/行为属性标签/MG/官网/过去14天浏览页面/过去14天浏览官网页面次数',\
            '全局标签/客户标签/行为属性标签/MG/APP/访问时长/过去14天访问APP平均时长',\
            '全局标签/客户标签/行为属性标签/MG/APP/积分/过去14天取得积分总数',\
            '全局标签/客户标签/行为属性标签/MG/APP/积分/总使用积分次数',\
            '全局标签/客户标签/行为属性标签/MG/APP/积分时间/最近一次使用积分时间',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/是否试乘试驾/是',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/MG销售相关总次数/MG总到店次数',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/次数/过去14天到店次数',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/APP/过去14天预约试乘试驾次数',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/是否打开短链/是',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/MG销售相关总次数/MG历史跟进次数',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/MG销售日期/MG上次外呼日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/销售相关日期/战败日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/最近一次进店时间/MG最近一次进店日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/客户综合信息/跨店建卡数量',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/销售相关日期/成交日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/MG销售日期/MG最近跟进日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/销售相关日期/最近一次试驾日期',\
            '主机厂专有标签/客户标签/行为属性标签/MG/活动/活动相关日期/最近参加活动日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/销售相关日期/最近短信发送日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/客户综合信息/线索渠道附加量',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/是否为首次购车/是',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/销售相关日期/MG最近一次建卡日期',\
            '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/是否变更经销商/是']

tag_name_sub_leaf = ['个人地址', '上次外呼结果', '线索来源', '性别', '年龄段', '购买车系', '试驾车系', '最近一次试驾车系',\
                     '留资车系', '战败核实原因', '战败核实状态', '战败原因（经销商）', 'MG战败核实结果', '最近短信发送结果', '成交经销商', '竞品品牌', '计划购车时间', '购车预算', \
                     '过去14天参加活动类型']

In [ ]:
# return result
[null_rate(mg_cust_6m, i) for i in tag_name_leaf]
[null_rate_contain(mg_cust_6m, i) for i in tag_name_sub_leaf]

#### Analysis of current CDP tags' distribution and deal conversion rate

In [ ]:
# MG Customers in 2019
cust_2019 = mg_tags_c[mg_tags_c['id'].isin(mg_tags_c[(mg_tags_c['tag_value'] == '客户寿命') & (mg_tags_c['score'] <= 498)\
                                         & (mg_tags_c['score'] >= 134)]['id'])]
cust_2019['split'] = cust_2019['tag_name'].str.split('/')

In [ ]:
pd.DataFrame(cust_2019[cust_2019['tag'] == '客户状态']['tag_value'].value_counts())

In [ ]:
status = cust_2019[cust_2019['tag'] == '客户状态'][['id', 'tag_value']]
status.rename(columns = {'tag_value': 'cust_status'}, inplace=True)

def sub_leaf_distribution(tag_name):
    targeted = cust_2019[cust_2019['tag'] == tag_name]
    targeted['value'] = [i[-2] for i in targeted['split']] 
    targeted = targeted[targeted['value'] != tag_name][['id','value']]
    return status.merge(targeted, how = 'left', on = 'id').groupby(['cust_status','value']).count()

def sub_leaf_distribution_2(tag_name):
    targeted = cust_2019[cust_2019['tag'] == tag_name]
    targeted['value'] = [i[-2] for i in targeted['split']] 
    return status.merge(targeted, how = 'left', on = 'id').groupby(['cust_status','value']).count()

def sub_leaf_distribution_3(tag_name, data, col):
    targeted = cust_2019[cust_2019['tag'] == tag_name][['id','tag_value']]
    return data.merge(targeted, how = 'left', on = 'id').groupby([col,'tag_value']).count()

def sub_leaf_distribution_4(tag_name, data, col):
    targeted = cust_2019[cust_2019['tag_name'] == tag_name][['id','score']]
    return data.merge(targeted, how = 'left', on = 'id').groupby([col,'score']).count()

In [ ]:
sub_leaf_distribution_4('主机厂专有标签/客户标签/业务属性标签/MG专有/销售/MG销售相关总次数/MG总到店次数', status, 'cust_status').to_csv('EDA_result/visit_vol.csv')

In [ ]:
car_leads = cust_2019[cust_2019['tag'] == '留资车系'][['id', 'tag_value']]
car_leads.rename(columns = {'tag_value': 'car_leads'}, inplace=True)
sub_leaf_distribution_3( '年龄段', car_leads, 'car_leads').to_csv('EDA_result/car_leads_age.csv')

In [ ]:
# MG customers distribution by city level
mg_city_level = cust_2019[cust_2019['tag'] == '个人地址']
mg_city_level['tag_value'].replace('上海市市辖区','上海市', inplace = True)
mg_city_level['tag_value'].replace('重庆市市辖区','重庆市', inplace = True)
mg_city_level['tag_value'].replace('北京市市辖区','北京市', inplace = True)
mg_city_level['tag_value'].replace('天津市市辖区','天津市', inplace = True)
mg_city_level['tag_value'].replace('重庆县','重庆市', inplace = True)
mg_city_level['tag_length'] = [len(i) for i in mg_city_level['split']]
mg_city_level = mg_city_level[mg_city_level['tag_length'] == 7].merge(city_level, how = 'left', \
                                                                     left_on = 'tag_value', right_on = 'city_name')

# city level
status.merge(mg_city_level[['id','city_level']], how = 'left', on = 'id').groupby(['cust_status','city_level']).count()

# city name
status.merge(mg_city_level[['id','city_name']], how = 'left', on = 'id').groupby(['cust_status','city_name']).count()

In [ ]:
# Distribution of last visit day
visit_date = cust_2019[cust_2019['tag_name'] == '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/最近一次进店时间/MG最近一次进店日期'][['id', 'score']]
visit_date['visit_date'] = [datetime(year=int(str(i)[0:4]), month=int(str(i)[4:6]), day=int(str(i)[6:8])).isoformat() \
                            for i in visit_date['score']]
visit_date['visit_weekday'] = [datetime.strptime(i, "%Y-%m-%dT%H:%M:%S").isoweekday() for i in visit_date['visit_date']]
visit_date[['id','visit_weekday']].groupby(['visit_weekday']).count()

In [ ]:
status.merge(visit_date[['id','visit_weekday']], how = 'left', on = 'id').groupby(['cust_status','visit_weekday']).count()

In [ ]:
# Funnel
leads_id = pd.DataFrame(cust_2019['id'].unique())
leads_id.rename(columns = {0: 'id'}, inplace=True)
trail_id = pd.DataFrame(cust_2019[cust_2019['tag_name'] == '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/是否试乘试驾/是']['id'])
visit_id = pd.DataFrame(cust_2019[cust_2019['tag_name'] == '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/MG销售相关总次数/MG总到店次数']['id'])
deal_id =  pd.DataFrame(cust_2019[cust_2019['tag_name'] == '主机厂专有标签/客户标签/业务属性标签/MG专有/销售/销售相关日期/成交日期']['id'])

In [ ]:
print('visit to trail:',len(visit_id.merge(trail_id, how = 'inner', on = 'id')['id'].unique()))
print('trail to deal:',len(trail_id.merge(deal_id, how = 'inner', on = 'id')['id'].unique()))
print('leads to trail:', len(leads_id[~leads_id['id'].isin(visit_id['id'])].merge(trail_id, how = 'inner', on = 'id')['id'].unique()))
print('leads to deal:', len(leads_id[~leads_id['id'].isin(pd.concat([visit_id['id'], trail_id['id']]))].merge(deal_id, how = 'inner', on = 'id')['id'].unique()))
print('visit to deal:', len(visit_id[~visit_id['id'].isin(trail_id['id'])].merge(deal_id, how = 'inner', on = 'id')['id'].unique()))

In [ ]:
[ i.shape for i in [leads_id, visit_id, trail_id, deal_id]]